In [2]:
import pandas as pd
import numpy as np
import plotly as plt

import math
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [3]:
freight = pd.read_csv("Data_Sets/avia_gor_nl__custom_3564729_monthly_linear.csv")
passengers = pd.read_csv("Data_Sets/avia_par_nl__custom_3564728_monthly_linear.csv")
#def clean(df):
#    df[["airp_country_1","airp_code_1","airp_country_2","airp_code_2"]] = df.airp_pr.str.split("_",expand=True)
#    df[["Year","Month"]] = df.TIME_PERIOD.str.split("-",expand=True)
#   cleaned = df
#   return cleaned

def top_5(df,n):
    df[["airp_country_1","airp_code_1","airp_country_2","airp_code_2"]] = df.airp_pr.str.split("_",expand=True)
    df[["Year","Month"]] = df.TIME_PERIOD.str.split("-",expand=True)
    top5 = df.groupby(["airp_country_2"]).sum().sort_values("OBS_VALUE", ascending= False).head(n)
    top5 = top5.reset_index()
    cleaned = df[df["airp_country_2"].isin(top5["airp_country_2"])]
    return cleaned

top_passengers = top_5(passengers,5)
top_passengers



,DATAFLOW,LAST UPDATE,freq,unit,tra_meas,airp_pr,TIME_PERIOD,OBS_VALUE,OBS_FLAG,airp_country_1,airp_code_1,airp_country_2,airp_code_2,Year,Month
4910,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_DE_EDDB,2011-01,11395,NaN,NL,EHAM,DE,EDDB,2011,01
4911,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_DE_EDDB,2011-02,11540,NaN,NL,EHAM,DE,EDDB,2011,02
4912,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_DE_EDDB,2011-03,13070,NaN,NL,EHAM,DE,EDDB,2011,03
4913,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_DE_EDDB,2011-04,13135,NaN,NL,EHAM,DE,EDDB,2011,04
4914,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_DE_EDDB,2011-05,13598,NaN,NL,EHAM,DE,EDDB,2011,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40384,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_US_KSLC,2022-03,15208,NaN,NL,EHAM,US,KSLC,2022,03
40385,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_US_KSLC,2022-04,19831,NaN,NL,EHAM,US,KSLC,2022,04
40386,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_US_KSLC,2022-05,22648,NaN,NL,EHAM,US,KSLC,2022,05
40387,ESTAT:AVIA_PAR_NL(1.0),23/09/22 23:00:00,M,PAS,PAS_CRD,NL_EHAM_US_KSLC,2022-06,22379,NaN,NL,EHAM,US,KSLC,2022,06


In [18]:
to_plot = top_passengers.groupby(['airp_country_2','TIME_PERIOD']).sum('OBS_VALUE').sort_values(by= 'TIME_PERIOD', ascending= True)


to_plot = to_plot.reset_index()
to_plot
#to_plot.reset_index().plot(x = 'TIME_PERIOD', y = 'OBS_VALUE')



,airp_country_2,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,DE,2000-01,170775,0.0
1,IT,2000-01,99824,0.0
2,UK,2000-01,445543,0.0
3,US,2000-01,297465,0.0
4,ES,2000-01,163101,0.0
...,...,...,...,...
1350,ES,2022-07,493600,0.0
1351,DE,2022-07,155860,0.0
1352,UK,2022-07,490787,0.0
1353,IT,2022-07,260973,0.0


In [24]:
fig = px.line(to_plot, x='TIME_PERIOD', y= 'OBS_VALUE' ,color='airp_country_2')
fig.show()